In [1]:
# import all the necessary libraries

import numpy as np
import pandas as pd
from datetime import datetime as dt
import itertools
from collections import defaultdict

# %matplotlib inline

In [2]:
# Read the data from a csv to a Pandas dataframe

data_loc = "C:\\Users\\matt\\jupyter-notebooks\\predicting-soccer\\data\\"

raw_data = []

raw_data.append(pd.read_csv(data_loc + '0607-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '0708-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '0809-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '0910-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '1011-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '1112-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '1213-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '1314-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '1415-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '1516-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '1617-E0.csv'))


In [3]:
# Parse date string data as time date format

def parse_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, "%d/%m/%y").date()
            
for year in raw_data:
    year.Date = year.Date.apply(parse_date)

In [4]:
# Filter the data points related to the gameplay

filtered_data = []
columns_req = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']

for year in raw_data:
    filtered_data.append(year[columns_req])

filtered_data[0].head(n=5)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,2006-08-19,Arsenal,Aston Villa,1,1,D
1,2006-08-19,Bolton,Tottenham,2,0,H
2,2006-08-19,Everton,Watford,2,1,H
3,2006-08-19,Newcastle,Wigan,2,1,H
4,2006-08-19,Portsmouth,Blackburn,3,0,H


In [5]:
# # Optimise the column naming structure for grouping using pandas groupby method

# for i, season in enumerate(filtered_data):
#     filtered_data[i].columns = [[    '', 'Home', 'Away', 'Home', 'Away',    ''],
#                                 ['Date', 'Team', 'Team',  'FTG',  'FTG', 'FTR']]

# filtered_data[0].head(n=5)


def init_teams(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    return teams

teams = init_teams(filtered_data[0])
print("Number of Teams:", len(teams))


Number of Teams: 20


In [11]:
def aggregate_scored(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    # the value corresponding to keys is a list containing the match location.
    for i in range(len(playing_stat)):
        HTGS = playing_stat.iloc[i]['FTHG']
        ATGS = playing_stat.iloc[i]['FTAG']
        teams[playing_stat.iloc[i].HomeTeam].append(HTGS)
        teams[playing_stat.iloc[i].AwayTeam].append(ATGS)
    
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    matchWeeks = int( len(playing_stat) / len(teams) * 2 ) + 1
    GoalsScored = pd.DataFrame(data=teams, index = [i for i in range(1,matchWeeks)]).T
    GoalsScored[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,matchWeeks):
        GoalsScored[i] = GoalsScored[i] + GoalsScored[i-1]
    return GoalsScored

def aggregate_conceded(playing_stat):
    # Create a dictionary with team names as keys
    teams = init_teams(playing_stat)
    
    # the value corresponding to keys is a list containing the match location.
    for i in range(len(playing_stat)):
        HTGS = playing_stat.iloc[i]['FTHG']
        ATGS = playing_stat.iloc[i]['FTAG']
        teams[playing_stat.iloc[i].HomeTeam].append(HTGS)
        teams[playing_stat.iloc[i].AwayTeam].append(ATGS)
    
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    matchWeeks = int( len(playing_stat) / len(teams) * 2 ) + 1
    GoalsScored = pd.DataFrame(data=teams, index = [i for i in range(1,matchWeeks)]).T
    GoalsScored[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,matchWeeks):
        GoalsScored[i] = GoalsScored[i] + GoalsScored[i-1]
    return GoalsScored

def aggregate_conceded(playing_stat):
    # Create a dictionary with team names as keys
    teams = init_teams(playing_stat)

    # the value corresponding to keys is a list containing the match location.
    for i in range(len(playing_stat)):
        ATGC = playing_stat.iloc[i]['FTHG']
        HTGC = playing_stat.iloc[i]['FTAG']
        teams[playing_stat.iloc[i].HomeTeam].append(HTGC)
        teams[playing_stat.iloc[i].AwayTeam].append(ATGC)

    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    matchWeeks = int( len(playing_stat) / len(teams) * 2 ) + 1
    GoalsConceded = pd.DataFrame(data=teams, index = [i for i in range(1, matchWeeks)]).T
    GoalsConceded[0] = 0

    # Aggregate to get uptil that point
    for i in range(2,matchWeeks):
        GoalsConceded[i] = GoalsConceded[i] + GoalsConceded[i-1]

    return GoalsConceded


def scored_and_conceded(season):
    scored = aggregate_scored(season)
    conceded = aggregate_conceded(season)

    j = 0
    HTGS = []
    ATGS = []
    HTGC = []
    ATGC = []
    
    for i in range(len(season)):
        ht = season.iloc[i].HomeTeam
        at = season.iloc[i].AwayTeam
        
        HTGS.append(scored.loc[ht][j])
        ATGS.append(scored.loc[at][j])
        HTGC.append(conceded.loc[ht][j])
        ATGC.append(conceded.loc[at][j])
        
        if ((i + 1)% 10) == 0:
            j = j + 1
        
    season['HTGS'] = HTGS
    season['ATGS'] = ATGS
    season['HTGC'] = HTGC
    season['ATGC'] = ATGC
    
    return season


# Apply to the filtered dataset
for i, season in enumerate(filtered_data):
    scored_and_conceded(filtered_data[i])

filtered_data[0].head(n=5)

c:\users\matt\envs\jupyter-notebook\lib\site-packages\ipykernel_launcher.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\matt\envs\jupyter-notebook\lib\site-packages\ipykernel_launcher.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\matt\envs\jupyter-notebook\lib\site-packages\ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTP,ATP,HM1,AM1,...,HM3,AM3,HM4,AM4,HM5,AM5,HTGS,ATGS,HTGC,ATGC
0,2006-08-19,Arsenal,Aston Villa,1,1,D,0,0,M,M,...,M,M,M,M,M,M,0,0,0,0
1,2006-08-19,Bolton,Tottenham,2,0,H,0,0,M,M,...,M,M,M,M,M,M,0,0,0,0
2,2006-08-19,Everton,Watford,2,1,H,0,0,M,M,...,M,M,M,M,M,M,0,0,0,0
3,2006-08-19,Newcastle,Wigan,2,1,H,0,0,M,M,...,M,M,M,M,M,M,0,0,0,0
4,2006-08-19,Portsmouth,Blackburn,3,0,H,0,0,M,M,...,M,M,M,M,M,M,0,0,0,0


In [12]:
# Get respective match points

def get_points(result):
    if result == 'W':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0
    

def get_cuml_points(matchres, matchWeeks, teamCount):
    matchres_points = matchres.applymap(get_points)
    for i in range(2, matchWeeks):
        matchres_points[i] = matchres_points[i] + matchres_points[i-1]
        
    matchres_points.insert(column = 0, loc = 0, value = [0*i for i in range(teamCount)])
    return matchres_points


def get_matchres(playing_stat, matchWeeks):
    # Create a dictionary with team names as keys
    teams = init_teams(playing_stat)

    # the value corresponding to keys is a list containing the match result
    for i in range(len(playing_stat)):
        if playing_stat.iloc[i].FTR == 'H':
            teams[playing_stat.iloc[i].HomeTeam].append('W')
            teams[playing_stat.iloc[i].AwayTeam].append('L')
        elif playing_stat.iloc[i].FTR == 'A':
            teams[playing_stat.iloc[i].AwayTeam].append('W')
            teams[playing_stat.iloc[i].HomeTeam].append('L')
        else:
            teams[playing_stat.iloc[i].AwayTeam].append('D')
            teams[playing_stat.iloc[i].HomeTeam].append('D')

    return pd.DataFrame(data=teams, index = [i for i in range(1, matchWeeks)]).T


def set_agg_points(playing_stat):
    teams = init_teams(playing_stat)
    matchWeeks = int( len(playing_stat) / len(teams) * 2 ) + 1
    
    matchres = get_matchres(playing_stat, matchWeeks)
    cum_pts = get_cuml_points(matchres, matchWeeks, len(teams))
    HTP = []
    ATP = []
    j = 0
    for i in range( len(playing_stat) ):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        HTP.append(cum_pts.loc[ht][j])
        ATP.append(cum_pts.loc[at][j])

        if ((i + 1)% 10) == 0:
            j = j + 1
            
    playing_stat['HTP'] = HTP
    playing_stat['ATP'] = ATP

    
# Apply to each dataset
for i, season in enumerate(filtered_data):
    set_agg_points(filtered_data[i])
    
filtered_data[0]

c:\users\matt\envs\jupyter-notebook\lib\site-packages\ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\matt\envs\jupyter-notebook\lib\site-packages\ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTP,ATP,HM1,AM1,...,HM3,AM3,HM4,AM4,HM5,AM5,HTGS,ATGS,HTGC,ATGC
0,2006-08-19,Arsenal,Aston Villa,1,1,D,0,0,M,M,...,M,M,M,M,M,M,0,0,0,0
1,2006-08-19,Bolton,Tottenham,2,0,H,0,0,M,M,...,M,M,M,M,M,M,0,0,0,0
2,2006-08-19,Everton,Watford,2,1,H,0,0,M,M,...,M,M,M,M,M,M,0,0,0,0
3,2006-08-19,Newcastle,Wigan,2,1,H,0,0,M,M,...,M,M,M,M,M,M,0,0,0,0
4,2006-08-19,Portsmouth,Blackburn,3,0,H,0,0,M,M,...,M,M,M,M,M,M,0,0,0,0
5,2006-08-19,Reading,Middlesbrough,3,2,H,0,0,M,M,...,M,M,M,M,M,M,0,0,0,0
6,2006-08-19,Sheffield United,Liverpool,1,1,D,0,0,M,M,...,M,M,M,M,M,M,0,0,0,0
7,2006-08-19,West Ham,Charlton,3,1,H,0,0,M,M,...,M,M,M,M,M,M,0,0,0,0
8,2006-08-20,Chelsea,Man City,3,0,H,0,0,M,M,...,M,M,M,M,M,M,0,0,0,0
9,2006-08-20,Man United,Fulham,5,1,H,0,0,M,M,...,M,M,M,M,M,M,0,0,0,0


In [13]:
# Set the Teams Form

def get_form(playing_stat,num):
    teams = 20
    matchWeeks = 39
    
    form = get_matchres(playing_stat, matchWeeks)
    form_final = form.copy()
    for i in range(num, matchWeeks):
        form_final[i] = ''
        j = 0
        while j < num:
            form_final[i] += form[i-j]
            j += 1           
    return form_final


def set_form(playing_stat, formCount=5):
    for num in range(1, formCount + 1):
        form = get_form(playing_stat,num)
        h = ['M' for i in range(num * 10)]  # since form is not available for n MW (n*10)
        a = ['M' for i in range(num * 10)]

        j = num
        for i in range((num*10), len(playing_stat)):
            ht = playing_stat.iloc[i].HomeTeam
            at = playing_stat.iloc[i].AwayTeam

            past = form.loc[ht][j]               # get past n results
            h.append(past[num-1])                    # 0 index is most recent

            past = form.loc[at][j]               # get past n results.
            a.append(past[num-1])                   # 0 index is most recent

            if ((i + 1)% 10) == 0:
                j = j + 1

        playing_stat['HM' + str(num)] = h                 
        playing_stat['AM' + str(num)] = a


# Apply to each dataset
for i, season in enumerate(filtered_data):
    set_form(filtered_data[i])
    
# Rearange the columns
cols = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3',
        'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5' ]

for i, season in enumerate(filtered_data):
    filtered_data[i] = filtered_data[i][cols]

filtered_data[0].tail(n=5)



c:\users\matt\envs\jupyter-notebook\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\matt\envs\jupyter-notebook\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC,...,HM1,HM2,HM3,HM4,HM5,AM1,AM2,AM3,AM4,AM5
375,2007-05-13,Middlesbrough,Fulham,3,1,H,41,37,48,57,...,W,L,D,L,L,W,L,D,L,L
376,2007-05-13,Portsmouth,Arsenal,0,0,D,45,63,42,35,...,L,W,D,W,L,D,W,D,W,W
377,2007-05-13,Sheffield United,Wigan,1,2,A,31,35,53,58,...,L,W,D,L,W,L,L,L,D,D
378,2007-05-13,Tottenham,Man City,2,1,H,55,28,53,42,...,D,W,W,D,D,L,L,D,L,D
379,2007-05-13,Watford,Newcastle,1,1,D,28,37,58,46,...,W,L,D,L,W,L,L,D,L,D


In [13]:
# Set the Matchweek

def set_week(playing_stat):
    j = 1
    week = []
    for i in range( len(playing_stat) ):
        week.append(j)
        if ((i + 1)% 10) == 0:
            j = j + 1
    playing_stat['MW'] = week


# Apply to each dataset
for i, season in enumerate(filtered_data):
    set_week(filtered_data[i])

filtered_data[0]

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC,...,HM2,HM3,HM4,HM5,AM1,AM2,AM3,AM4,AM5,MW
0,2006-08-19,Arsenal,Aston Villa,1,1,D,0,0,0,0,...,M,M,M,M,M,M,M,M,M,1
1,2006-08-19,Bolton,Tottenham,2,0,H,0,0,0,0,...,M,M,M,M,M,M,M,M,M,1
2,2006-08-19,Everton,Watford,2,1,H,0,0,0,0,...,M,M,M,M,M,M,M,M,M,1
3,2006-08-19,Newcastle,Wigan,2,1,H,0,0,0,0,...,M,M,M,M,M,M,M,M,M,1
4,2006-08-19,Portsmouth,Blackburn,3,0,H,0,0,0,0,...,M,M,M,M,M,M,M,M,M,1
5,2006-08-19,Reading,Middlesbrough,3,2,H,0,0,0,0,...,M,M,M,M,M,M,M,M,M,1
6,2006-08-19,Sheffield United,Liverpool,1,1,D,0,0,0,0,...,M,M,M,M,M,M,M,M,M,1
7,2006-08-19,West Ham,Charlton,3,1,H,0,0,0,0,...,M,M,M,M,M,M,M,M,M,1
8,2006-08-20,Chelsea,Man City,3,0,H,0,0,0,0,...,M,M,M,M,M,M,M,M,M,1
9,2006-08-20,Man United,Fulham,5,1,H,0,0,0,0,...,M,M,M,M,M,M,M,M,M,1


In [17]:
# Export to CSV for processing
pd.concat(filtered_data).to_csv(data_loc + "..\\final_dataset.csv")
# df.to_csv(loc + "final_dataset.csv")